In [1]:
import uproot
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle

import os
import sys
sys.path.append(os.path.abspath(".."))


In [2]:
with open("../intermediate_files/test_charge.pkl", "rb") as f:
    outputs = pickle.load(f)

true_gamma_info_df = outputs[0]
downsampled_Tcluster_spacepoints_with_charge = outputs[1]
real_nu_reco_nu_downsampled_spacepoints = outputs[2]
real_nu_reco_cosmic_downsampled_spacepoints = outputs[3]
real_cosmic_reco_nu_downsampled_spacepoints = outputs[4]
real_cosmic_reco_cosmic_downsampled_spacepoints = outputs[5]
real_gamma1_downsampled_spacepoints = outputs[6]
real_gamma2_downsampled_spacepoints = outputs[7]
real_other_particles_downsampled_spacepoints = outputs[8]
real_cosmic_downsampled_spacepoints = outputs[9]


In [3]:
event_index = 0

for i in range(len(outputs)):
    if i == 0: # true gamma info df
        continue
    print(len(outputs[i][event_index]))

2030
654
3
132
1241
115
0
544
1371


In [4]:
reco_nu_vtx = []
true_nu_vtx = []
true_gamma1_pairconvert_vtx = []
true_gamma2_pairconvert_vtx = []

for event_i in range(len(true_gamma_info_df)):

    reco_nu_vtx.append([true_gamma_info_df["reco_nu_vtx_x"][event_i], true_gamma_info_df["reco_nu_vtx_y"][event_i], true_gamma_info_df["reco_nu_vtx_z"][event_i]])
    true_nu_vtx.append([true_gamma_info_df["true_nu_vtx_x"][event_i], true_gamma_info_df["true_nu_vtx_y"][event_i], true_gamma_info_df["true_nu_vtx_z"][event_i]])
   
    if len(true_gamma_info_df["true_gamma_pairconversion_xs"][event_i]) > 0:
        true_gamma1_pairconvert_vtx.append([true_gamma_info_df["true_gamma_pairconversion_xs"][event_i][0], true_gamma_info_df["true_gamma_pairconversion_ys"][event_i][0], true_gamma_info_df["true_gamma_pairconversion_zs"][event_i][0]])
    else:
        true_gamma1_pairconvert_vtx.append(None)
    if len(true_gamma_info_df["true_gamma_pairconversion_xs"][event_i]) > 1:
        true_gamma2_pairconvert_vtx.append([true_gamma_info_df["true_gamma_pairconversion_xs"][event_i][1], true_gamma_info_df["true_gamma_pairconversion_ys"][event_i][1], true_gamma_info_df["true_gamma_pairconversion_zs"][event_i][1]])
    else:
        true_gamma2_pairconvert_vtx.append(None)


if len(true_gamma1_pairconvert_vtx) == 0:
    true_gamma1_pairconvert_vtx = None
if len(true_gamma2_pairconvert_vtx) == 0:
    true_gamma2_pairconvert_vtx = None


In [5]:
from helpers.plotting_3d import plot_event

# event that we failed before
event_index = 61

event_index = 2

#event_index = 10084

points_dic = {
    "Tcluster_spacepoints_with_charge": (downsampled_Tcluster_spacepoints_with_charge[event_index][:, :3], downsampled_Tcluster_spacepoints_with_charge[event_index][:, 3], "jet", 2, True),
    "real_gamma1_downsampled_spacepoints": (real_gamma1_downsampled_spacepoints[event_index], "green", None, 2, True),
    "real_gamma2_downsampled_spacepoints": (real_gamma2_downsampled_spacepoints[event_index], "lightgreen", None, 2, True),
    "real_other_particles_downsampled_spacepoints": (real_other_particles_downsampled_spacepoints[event_index], "brown", None, 2, True),
    "real_cosmic_downsampled_spacepoints": (real_cosmic_downsampled_spacepoints[event_index], "blue", None, 2, True),
    "reco_nu_vtx": (reco_nu_vtx[event_index], "orange", None, 10, "legendonly"),
    "true_nu_vtx": (true_nu_vtx[event_index], "red", None, 10, "legendonly"),
    "true_gamma1_pairconvert_vtx": (true_gamma1_pairconvert_vtx[event_index], "green", None, 10, True),
    "true_gamma2_pairconvert_vtx": (true_gamma2_pairconvert_vtx[event_index], "lightgreen", None, 10, True),
}

plot_event(event_index, points_dic)


name = Tcluster_spacepoints_with_charge, points.shape = (586, 3)
name = real_gamma1_downsampled_spacepoints, points.shape = (96, 4)
name = real_gamma2_downsampled_spacepoints, points.shape = (0, 4)
name = real_other_particles_downsampled_spacepoints, points.shape = (2, 4)
name = real_cosmic_downsampled_spacepoints, points.shape = (488, 4)
name = reco_nu_vtx, points.shape = (3,)
name = true_nu_vtx, points.shape = (3,)
name = true_gamma1_pairconvert_vtx, points.shape = (3,)
name = true_gamma2_pairconvert_vtx, points.shape = ()
